In [29]:
import json
import time
from typing import Dict, List

import requests

BASE_URL = "https://api.ai71.ai/v1/"

In [30]:
API_KEY = "ai71-api-b1e07fa1-d007-41cd-8306-85fc952e12a6"

In [31]:
def check_budget():
    resp = requests.get(
        f"{BASE_URL}check_budget",
        headers={"Authorization": f"Bearer {API_KEY}"},
    )
    resp.raise_for_status()
    print(json.dumps(resp.json(), indent=4))

In [40]:
check_budget()

{
    "remaining_budget": 9995
}


In [33]:
def bulk_generate(n_questions: int, question_categorizations: List[Dict], user_categorizations: List[Dict]):
    resp = requests.post(
        f"{BASE_URL}bulk_generation",
        headers={"Authorization": f"Bearer {API_KEY}"},
        json={
                "n_questions": n_questions,
                "question_categorizations": question_categorizations,
                "user_categorizations": user_categorizations
            }
    )
    resp.raise_for_status()
    request_id = resp.json()["request_id"]
    print(json.dumps(resp.json(), indent=4))

    result = wait_for_generation_to_finish(request_id)
    return result


def wait_for_generation_to_finish(request_id: str):
    while True:
        resp = requests.get(
            f"{BASE_URL}fetch_generation_results",
            headers={"Authorization": f"Bearer {API_KEY}"},
            params={"request_id": request_id},
        )
        resp.raise_for_status()
        if resp.json()["status"] == "completed":
            print(json.dumps(resp.json(), indent=4))
            return resp.json()
        else:
            print("Waiting for generation to finish...")
            time.sleep(5)

In [34]:
question_length_categorization = {
    "categorization_name": "question_length",
    "categories": [
        {
            "name": "short",
            "description": "a short question with no more than 6 words.",
            "probability": 0.4
        },
        {
            "name": "long",
            "description": "a long question with at least 7 words.",
            "probability": 0.6
        }
    ]
}

question_formulation_categorization = {
    "categorization_name": "question_formulation",
    "categories": [
        {
            "name": "natural",
            "description": "phrased in the way people typically speak, reflecting everyday language use, without formal or artificial structure.",
            "probability": 0.8
        },
        {
            "name": "search query",
            "description": "phrased as a typed web query for search engines (only keywords, without punctuation and without a natural-sounding structure).",
            "probability": 0.2
        }
    ]
}

user_expertise_categorization = {
    "categorization_name": "user_expertise",
    "categories": [
        {
            "name": "expert",
            "description": "an expert of the subject discussed in the document, therefore he asks complex questions.",
            "probability": 0.8
        },
        {
        "name": "common person",
            "description": "a common person who is not expert of the subject discussed in the document, therefore he asks basic questions.",
            "probability": 0.2
        }
    ]
}

multi_doc_categorization = {
    "categorization_name": "multi-doc",
    "categories": [
        {
            "name": "comparison",
            "description": "a comparison question that requires comparing two related concepts or entities. The comparison must be natural and reasonable, i.e., comparing two entities by a common attribute which is meaningful and relevant to both entities. For example: 'Who is older, Glenn Hughes or Ross Lynch?', 'Are Pizhou and Jiujiang in the same province?', 'Pyotr Ilyich Tchaikovsky and Giuseppe Verdi have this profession in common'. The information required to answer the question needs to come from two documents, specifically, the first document must provide information about the first entity/concept, while the second must provide information about the second entity/concept.",
            "probability": 0.5,
            "is_multi_doc": True
        },
        {
            "name": "multi-aspect",
            "description": "A question about two different aspects of the same entity/concept. For example: 'What are the advantages of AI-powered diagnostics, and what are the associated risks of bias in medical decision-making?', 'How do cryptocurrencies enable financial inclusion, and what are the security risks associated with them?'. The information required to answer the question needs to come from two documents, specifically, the first document must provide information about the first aspect, while the second must provide information about the second aspect.",
            "probability": 0.5,
            "is_multi_doc": True
        }
    ]
}

In [41]:
results = bulk_generate(n_questions=10,
                         question_categorizations=[question_length_categorization, question_formulation_categorization],
                         user_categorizations=[user_expertise_categorization]
                         )

HTTPError: 429 Client Error: Too Many Requests for url: https://api.ai71.ai/v1/bulk_generation

In [42]:
result = wait_for_generation_to_finish("598d3657-845e-4f05-abf5-a86192a1d2b8")
print(result)

HTTPError: 500 Server Error: Internal Server Error for url: https://api.ai71.ai/v1/fetch_generation_results?request_id=94488cb2-b93d-439f-8259-3c8c280ad44d